# Train the Convolutional auto encoder

This notebook defines and trains the CAE model. 

The NVIDIA Docker container TensorFlow 21.02-tf2 was used to generate a computational environment.

In [ ]:
import numpy as np
from tensorflow.keras.layers import Conv2D, UpSampling2D, BatchNormalization
from tensorflow.keras.models import Sequential
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
import tensorflow as tf

In [ ]:
%%time
tensors = np.load('sentinel_gb_995_3_20210317.npz')['tensors']
random = np.random.random(len(tensors))
tensors = tensors[random.argsort()]
a = np.percentile(random,80)
x_train = tensors[random<=a]
x_test = tensors[random>a]
#x_tensors = tensors

In [ ]:
print(tensors.shape,tensors.dtype)

In [ ]:
tensors[0,0]

In [ ]:
# The next three methods to visualize input/output of our model side-by-side
def hstackimgs(min, max, images):
    return np.hstack(images[i] for i in range(min, max))

def sqstackimgs(length, height, images):
    return np.vstack(hstackimgs(i*length, (i+1)*length, images) for i in range(height))

def sbscompare(images1, images2, length, height):
    A = sqstackimgs(length, height, images1)
    B = sqstackimgs(length, height, images2)
    C = np.ones((A.shape[0], 32, 3))
    return np.hstack((A, C, B))

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard('.',histogram_freq=1)

In [ ]:
# Symmetrical
model = Sequential()
k = 4
s = 16
pad = 'same'

model.add(Conv2D(s, kernel_size=k, strides=1, padding=pad, activation='relu', input_shape=(s, s, 4)))
model.add(Conv2D(s, kernel_size=k, strides=1, padding=pad, activation='relu'))      # 16x16x16
model.add(BatchNormalization())     # 16x16x16
model.add(Conv2D(s, kernel_size=k, strides=2, padding=pad, activation='relu'))      # 8x8x16
model.add(Conv2D(s, kernel_size=k, strides=1, padding=pad, activation='relu'))      # 8x8x16
model.add(Conv2D(4, kernel_size=k, strides=1, padding=pad, activation='relu'))      # 8x8x4
model.add(BatchNormalization())     # 8x8x4
model.add(Conv2D(4, kernel_size=k, strides=1, padding=pad, activation='relu'))      # 8x8x4
model.add(Conv2D(1, kernel_size=k, strides=1, padding=pad, activation='relu'))      # 8x8x1
model.add(BatchNormalization())     # 8x8x1
model.add(Conv2D(1, kernel_size=k, strides=1, padding=pad, activation='relu'))      # 8x8x1
model.add(Conv2D(4, kernel_size=k, strides=1, padding=pad, activation='relu'))      # 8x8x4
model.add(BatchNormalization())     # 8x8x4
model.add(Conv2D(4, kernel_size=k, strides=1, padding=pad, activation='relu'))      # 8x8x4
model.add(Conv2D(s, kernel_size=k, strides=1, padding=pad, activation='relu'))      # 8x8x16
model.add(Conv2D(s, kernel_size=k, strides=1, padding=pad, activation='relu'))      # 8x8x16
model.add(UpSampling2D())
model.add(Conv2D(s, kernel_size=k, strides=1, padding=pad, activation='relu'))      # 16x16x16
model.add(Conv2D(s, kernel_size=k, strides=1, padding=pad, activation='relu'))      # 16x16x16
model.add(BatchNormalization())
model.add(Conv2D(4,  kernel_size=1, strides=1, padding=pad, activation='sigmoid'))   # 16x16x4

model.compile(optimizer='adam', metrics=['accuracy'], loss='mean_squared_error')
model.summary()

In [ ]:
# Fit the model
model.fit(x_train, x_train, epochs=100, batch_size=100,callbacks=[tensorboard_callback],validation_data=(x_test, x_test))

In [ ]:
x_test = x_test[:400]
pred_imgs = model.predict(x_test)

In [ ]:
plt.imshow(sbscompare(x_test[:,:,:,0:3], pred_imgs[:,:,:,0:3], 20, 20))
plt.axis('off')
plt.rcParams["figure.figsize"] = [60,60]
plt.show()

In [ ]:
plt.imshow(sbscompare(x_test[:,:,:,1:4], pred_imgs[:,:,:,1:4], 20, 20))
plt.axis('off')
plt.rcParams["figure.figsize"] = [60,60]
plt.show()

In [ ]:
layer_name = model.layers[9].name
print(layer_name)

In [ ]:
%%time
intermediate_layer_model = Model(inputs=model.input,
                                 outputs=model.get_layer(layer_name).output)

In [ ]:
%%time
intermediate_output = intermediate_layer_model(x_test[:1000])

In [ ]:
intermediate_output

In [ ]:
gen = intermediate_output.reshape(-1,64)

In [ ]:
gen.shape

In [ ]:
%%time
np.savez_compressed("sentinel_flattened_gb_3_20210307",gen=gen)

In [ ]:
model.save("models/gb_postcode_3")

In [ ]:
intermediate_layer_model.save("models/gb_postcode_encoder_3")